In [247]:
import pandas as pd
import numpy as np

In [248]:
parse_dates=['date']
df= pd.read_csv('step2.csv',parse_dates=parse_dates)
df_copy = df.copy()
df

,webClientID,UserID,User_Vintage,date,datetime,ProductID,Activity,OS
0,WI100000112772,U106593,260,2018-05-15,2018-05-15 06:48:15,PR100017,CLICK,ANDROID
1,WI100000112772,U106593,260,2018-05-16,2018-05-16 06:53:23,PR100241,PAGELOAD,ANDROID
2,WI100000112772,U106593,260,2018-05-15,2018-05-15 06:47:49,PR100017,CLICK,ANDROID
3,WI100000112772,U106593,260,2018-05-16,2018-05-16 06:54:26,PR100901,PAGELOAD,ANDROID
4,WI100000112772,U106593,260,2018-05-15,2018-05-15 06:47:36,PR100017,CLICK,ANDROID
...,...,...,...,...,...,...,...,...
584217,WI100000518890,U120504,518,2018-05-20,2018-05-20 10:09:39,PR100430,PAGELOAD,ANDROID
584218,WI100000977958,U121518,632,2018-05-19,2018-05-19 12:51:25,PR100083,PAGELOAD,MAC OS X
584219,WI100000696937,U119259,52,2018-05-21,2018-05-21 06:49:59,PR100102,PAGELOAD,WINDOWS
584220,WI100000949074,U127061,89,2018-05-14,2018-05-14 22:27:00,PR100294,CLICK,ANDROID


In [249]:
df.datetime = pd.to_datetime(df.datetime)

##### We are supposed to generate an input feature set as on 28-May-2018. So, the data is from 07-May-2018 to 27-May-2018. Then sorting it based on datetime

In [250]:
df = df.sort_values(by=['datetime'], ascending=True).reset_index()

In [251]:
df.date.unique()

array(['2018-05-07T00:00:00.000000000', '2018-05-08T00:00:00.000000000',
       '2018-05-09T00:00:00.000000000', '2018-05-10T00:00:00.000000000',
       '2018-05-11T00:00:00.000000000', '2018-05-12T00:00:00.000000000',
       '2018-05-13T00:00:00.000000000', '2018-05-14T00:00:00.000000000',
       '2018-05-15T00:00:00.000000000', '2018-05-16T00:00:00.000000000',
       '2018-05-17T00:00:00.000000000', '2018-05-18T00:00:00.000000000',
       '2018-05-19T00:00:00.000000000', '2018-05-20T00:00:00.000000000',
       '2018-05-21T00:00:00.000000000', '2018-05-22T00:00:00.000000000',
       '2018-05-23T00:00:00.000000000', '2018-05-24T00:00:00.000000000',
       '2018-05-25T00:00:00.000000000', '2018-05-26T00:00:00.000000000',
       '2018-05-27T00:00:00.000000000',                           'NaT'],
      dtype='datetime64[ns]')

### Calculating dictionary for User_Vintage and Most_Active_OS

In [252]:
User_Vintage = df_copy.groupby(['UserID'])['User_Vintage'].mean().to_dict()

In [305]:
#Most_Active_OS = df_copy.groupby(['UserID'])['OS'].agg(lambda x: x.mode() if type(x.mode())==str else x.tail(1)).to_dict()
Most_Active_OS = df_copy.groupby(['UserID'])['OS'].agg(lambda x: x.mode()).to_dict()

### Creating a count column based on the number of each users in the table.

In [254]:
df['count'] = df.UserID.map(df.UserID.value_counts().to_dict())

### Filtering null DateTime

Creating an another count1 column to find the number of each users in this filtered table. The idea is to again filter out the those unique users with null dateTime values that aren't present in our dataframe with non-null dateTime values.

Count - Represents the total count of a user data in main table

Count1 - Represents the total count of a user data in this section of filtered table

The rows where count - count1 is 0, are the rows with users that will be completely out of the  dataframe with non-null dateTime values, which we will use later in this notebook.

The purpose of this calculation is to find the recently viewed products. Since these users have null dateTime value and like a very few records (in many cases, only 1), then that's their recently viewed product.

In [255]:
temp = df[pd.isnull(df['date'])]
temp['count1'] = temp.UserID.map(temp.UserID.value_counts().to_dict())
temp['count-count1'] = temp['count']- temp['count1']

C:\Users\Ryuk\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Ryuk\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [256]:
temp1 = temp[temp['count-count1'] == 0]
temp1 = temp1[pd.notnull(temp1['ProductID'])]
temp1= temp1[temp1['Activity'] == 'PAGELOAD']
RVP1 = temp1.groupby(['UserID'])['ProductID'].agg(lambda x: x.tail(1) ).to_dict()

##### Creating a dictionary by the name of RVP1 for Recently_Viewed_Product

### Removing all null DateTime rows for further calculation

In [257]:
df = df.loc[pd.notnull(df.datetime)]

### Imputing missing ProductID and Activity

##### Creating two columns based on taking the difference of dateTime value. Taking difference from previous rows (back_diff) and the next rows (forward_diff).

In [258]:
df['back_diff'] = df.groupby('UserID')['datetime'].apply(lambda x: x.diff())

In [259]:
df['forward_diff'] = df.groupby('UserID')['back_diff'].apply(lambda x: x.shift(-1, axis = 0))

##### Doing the forward and backward fill for ProductID in two separate columns.

In [260]:
df['ffill_ProductID'] = df.groupby('UserID')['ProductID'].transform(lambda x: x.ffill())
df['bfill_ProductID'] = df.groupby('UserID')['ProductID'].transform(lambda x: x.bfill())

The idea is where the ffill_ProductID is equal to bfill_ProductID, then impute any as they are same.
When they arent equal, we check the back_diff and forward_diff and impute whether ffill_ProductID or bfill_ProductID.
The idea is if a productID is missing, it will likely be the preceding one if there is a very small difference in time with the preceding time compared to the following difference. 
Example- If I viewed product A and 5 mins later viewed another product with missing ID, then 1hr later viewed product B, then that missing value is likely to be a product A.

In [261]:
df.loc[df['ProductID'].isnull() & df['back_diff'].isnull(),'ProductID'] = df.loc[df['ProductID'].isnull() & df['back_diff'].isnull()]['bfill_ProductID'].apply(lambda x: x)

In [262]:
df.loc[df['ProductID'].isnull() & df['forward_diff'].isnull(),'ProductID'] = df.loc[df['ProductID'].isnull() & df['forward_diff'].isnull()]['ffill_ProductID'].apply(lambda x: x)

In [263]:
df.loc[df['ProductID'].isnull() & df['back_diff'].notnull() & df['forward_diff'].notnull(),'ProductID']= df.loc[df['ProductID'].isnull() & df['back_diff'].notnull() & df['forward_diff'].notnull()][['back_diff','forward_diff','ffill_ProductID','bfill_ProductID']].apply(lambda x: x['ffill_ProductID'] if x['back_diff']<= x['forward_diff'] else x['bfill_ProductID'],  axis=1 )

In [264]:
df.loc[df['ProductID'].isnull() & df['ffill_ProductID'].notnull(),'ProductID']= df.loc[df['ProductID'].isnull() & df['ffill_ProductID'].notnull() ]['ffill_ProductID'].apply(lambda x: x )
df.loc[df['ProductID'].isnull() & df['bfill_ProductID'].notnull(),'ProductID']= df.loc[df['ProductID'].isnull() & df['bfill_ProductID'].notnull() ]['bfill_ProductID'].apply(lambda x: x )

##### Doing the forward and backward fill for Activity.

In [265]:
df['Activity'] = df.groupby('UserID')['Activity'].transform(lambda x: x.ffill())
df['Activity'] = df.groupby('UserID')['Activity'].transform(lambda x: x.bfill())

##### Creating a dictionary by the name of RVP2 for Recently_Viewed_Product

In [266]:
RVP2 =df[df.Activity =='PAGELOAD'].groupby(['UserID'])['ProductID'].agg(lambda x: x.tail(1) ).to_dict()

##### Merging both RVP1 and RVP2 to create RVP for Recently_Viewed_Product

In [267]:
RVP = dict(RVP1)
RVP.update(RVP2)
RVP

{'U100042': 'PR100182',
 'U100283': 'PR100850',
 'U100723': 'PR100086',
 'U100783': 'PR100792',
 'U100982': 'PR100102',
 'U101162': 'PR100301',
 'U101416': 'PR101463',
 'U101494': 'PR100102',
 'U101577': 'PR100527',
 'U102213': 'PR100234',
 'U102294': 'PR100360',
 'U102398': 'PR100023',
 'U102426': 'PR100017',
 'U103149': 'PR100070',
 'U103164': 'PR101004',
 'U103213': 'PR100321',
 'U103279': 'PR100017',
 'U103363': 'PR100413',
 'U103374': 'PR101052',
 'U103399': 'PR102009',
 'U103613': 'PR100340',
 'U103660': 'PR100877',
 'U103712': 'PR100001',
 'U103728': 'PR100470',
 'U103796': 'PR100184',
 'U103888': 'PR100562',
 'U104335': 'PR100406',
 'U104426': 'PR100172',
 'U104442': 'PR100901',
 'U104506': 'PR100102',
 'U104564': 'PR100918',
 'U104593': 'PR101071',
 'U104689': 'PR100152',
 'U104713': 'PR100446',
 'U104797': 'PR100873',
 'U105066': 'PR102372',
 'U105577': 'PR100401',
 'U105637': 'PR100330',
 'U105902': 'PR101314',
 'U106013': 'PR100405',
 'U106178': 'PR100872',
 'U106204': 'PR1

### Filtering last_15_days 

In [268]:
df_15 = df[df["datetime"] >= pd.to_datetime("2018-05-13")]
df_15 = df_15[df_15["datetime"] <= pd.to_datetime("2018-05-27")]

In [269]:
df_15

,index,webClientID,UserID,User_Vintage,date,datetime,ProductID,Activity,OS,count,back_diff,forward_diff,ffill_ProductID,bfill_ProductID
163630,68067,WI10000014966,U122747,175,2018-05-13,2018-05-13 00:00:07,PR100070,CLICK,WINDOWS,258,0 days 00:00:10,0 days 00:00:00,PR100070,PR100070
163631,68164,WI10000014966,U122747,175,2018-05-13,2018-05-13 00:00:07,PR100070,CLICK,WINDOWS,258,0 days 00:00:00,0 days 00:00:00,PR100070,PR100070
163632,68028,WI10000014966,U122747,175,2018-05-13,2018-05-13 00:00:07,PR100070,CLICK,WINDOWS,258,0 days 00:00:00,0 days 00:00:05,PR100070,PR100070
163633,68056,WI10000014966,U122747,175,2018-05-13,2018-05-13 00:00:12,PR100070,CLICK,WINDOWS,258,0 days 00:00:05,0 days 00:00:00,PR100070,PR100070
163634,68118,WI10000014966,U122747,175,2018-05-13,2018-05-13 00:00:12,PR100070,CLICK,WINDOWS,258,0 days 00:00:00,0 days 00:00:20,PR100070,PR100070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519978,380146,WI1000009775,U104291,101,2018-05-26,2018-05-26 23:55:43,PR100166,CLICK,WINDOWS,58,0 days 09:21:26,0 days 00:00:06,PR100166,PR100166
519979,380179,WI1000009775,U104291,101,2018-05-26,2018-05-26 23:55:49,PR100166,PAGELOAD,WINDOWS,58,0 days 00:00:06,0 days 00:00:06,PR100166,PR100166
519980,380137,WI1000009775,U104291,101,2018-05-26,2018-05-26 23:55:55,PR100166,CLICK,WINDOWS,58,0 days 00:00:06,0 days 14:30:47,PR100166,PR100166
519981,360387,WI10000031569,U105266,720,2018-05-26,2018-05-26 23:56:00,PR100574,PAGELOAD,WINDOWS,123,0 days 00:46:05,0 days 00:05:22,PR100574,PR100574


In [270]:
No_Of_Products_Viewed_15_Days = df_15.groupby(['UserID'])['ProductID'].nunique().to_dict()

In [271]:
df_15 = df_15[df_15.Activity == 'PAGELOAD']
df_15 = df_15.loc[pd.notnull(df_15.ProductID)]

In [272]:
df_15.groupby(['UserID'])['ProductID'].agg(lambda x: x.mode()) 

UserID
U100002                          PR100258
U100003              [PR100079, PR100178]
U100004    [PR100069, PR100197, PR100753]
U100005                          PR100234
U100006                          PR101111
                        ...              
U129366                          PR100927
U129367                          PR100156
U129369                          PR100507
U129370                          PR100739
U129371                          PR100051
Name: ProductID, Length: 21581, dtype: object

In [273]:
Most_Viewed_product_15_Days = df_15.groupby(['UserID'])['ProductID'].agg(lambda x: x.mode() if type(x.mode())==str else x.tail(1) ).to_dict()


### Filtering last_7_days

In [274]:
df_7 = df[df["datetime"] >= pd.to_datetime("2018-05-21")]
df_7 = df_7[df_7["datetime"] <= pd.to_datetime("2018-05-27")]

In [275]:
No_of_days_Visited_7_Days = df_7.groupby(['UserID'])['date'].nunique().to_dict()

In [276]:
Pageloads_last_7_days = df_7[df_7.Activity == 'PAGELOAD'].groupby(['UserID'])['Activity'].count().to_dict()

In [277]:
Clicks_last_7_days = df_7[df_7.Activity == 'CLICK'].groupby(['UserID'])['Activity'].count().to_dict()

### Loading sample_submission file

In [278]:
df_= pd.read_csv('sample_submission.csv')

### Modifying columns by mapping with out dictionaries

In [296]:
df_.No_of_days_Visited_7_Days = df_.UserID.map(No_of_days_Visited_7_Days)

In [297]:
df_.No_Of_Products_Viewed_15_Days = df_.UserID.map(No_Of_Products_Viewed_15_Days)

In [298]:
df_.User_Vintage = df_.UserID.map(User_Vintage) 

In [299]:
df_.Most_Viewed_product_15_Days = df_.UserID.map(Most_Viewed_product_15_Days)

In [306]:
df_.Most_Active_OS = df_.UserID.map(Most_Active_OS)

In [301]:
df_.Recently_Viewed_Product = df_.UserID.map(RVP)

In [302]:
df_.Pageloads_last_7_days = df_.UserID.map(Pageloads_last_7_days)

In [303]:
df_.Clicks_last_7_days = df_.UserID.map(Clicks_last_7_days)

### Filling null with appropriate values

In [308]:
df_['No_of_days_Visited_7_Days'] = df_['No_of_days_Visited_7_Days'].fillna(0)
df_['No_Of_Products_Viewed_15_Days'] = df_['No_Of_Products_Viewed_15_Days'].fillna(0)
df_['Most_Viewed_product_15_Days'] = df_['Most_Viewed_product_15_Days'].fillna('Product101')
df_['Recently_Viewed_Product'] = df_['Recently_Viewed_Product'].fillna('Product101')
df_['Pageloads_last_7_days'] = df_['Pageloads_last_7_days'].fillna(0)
df_['Clicks_last_7_days'] = df_['Clicks_last_7_days'].fillna(0)

### Dealing with cases, probably not required

In [288]:
#df_.loc[df_['Most_Viewed_product_15_Days']!='Product101','Most_Viewed_product_15_Days' ] = df_.loc[df_['Most_Viewed_product_15_Days']!='Product101','Most_Viewed_product_15_Days' ].str.replace('PR','Pr')
#df_.loc[df_['Recently_Viewed_Product']!='Product101','Recently_Viewed_Product' ] = df_.loc[df_['Recently_Viewed_Product']!='Product101','Recently_Viewed_Product' ].str.replace('PR','Pr')

### Finally TheOuput file.

In [309]:
df_.to_csv('TheOutput.csv', index=False) 

In [294]:
output = pd.read_csv('TheOutput.csv')
output

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,0.0,2.0,53,PR100258,ANDROID,PR100258,0.0,0.0
1,U100003,1.0,2.0,1021,PR100079,WINDOWS,PR100079,1.0,2.0
2,U100004,1.0,15.0,341,PR100069,WINDOWS,PR100069,1.0,0.0
3,U100005,1.0,3.0,681,PR100234,ANDROID,PR100234,2.0,0.0
4,U100006,1.0,1.0,55,PR101111,ANDROID,PR101111,1.0,0.0
...,...,...,...,...,...,...,...,...,...
34045,U136960,0.0,0.0,755,Product101,WINDOWS,Product101,0.0,0.0
34046,U136961,0.0,0.0,60,Product101,ANDROID,PR101381,0.0,0.0
34047,U136963,0.0,0.0,1420,Product101,ANDROID,PR100166,0.0,0.0
34048,U136964,0.0,0.0,495,Product101,WINDOWS,Product101,0.0,0.0
